# Arima Modelling on BQML for Demand Forecasting

## Loading necessary libraries 

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
import seaborn as sns
import pandas as pd
import math
import numpy as np
import numpy as papermill
from google.cloud import bigquery
import warnings; warnings.simplefilter('ignore')
client = bigquery.Client()

for first time install pandas_gbq to read data from BQ to a pandas dataframe via pandas.from_gbq

In [ ]:
!pip install pandas_gbq
!pip install google-cloud-bigquery-storage

In [ ]:
PROJECT_ID = ''
DATASET_NAME = '' # BQ dataset that has the Data Table and will store the tained Model
MODEL_NAME = '' # Name name ofg the model we are going to train
TABLE_NAME = '' # BQ Table name with the data
TRAIN_UNTIL_DATETIME = '2019-07-21' # The date up to wich the Arima Model will be trained.
FORECAST_HORIZON = 14 # Number of dates in the future the model will forecast - Used in Forecasting, not in training

TIME_SERIES_ID_COLUMN = '' # Table column or the forecast unit - defining this builds a timeseries model for each ID column. i.e a forecast per SKU
TIME_SERIES_DATA_COLUMN = '' # Table column of the attribute we want to be able forecast 
TIME_SERIES_TIMESTAMP_COLUMN ='' # Table column that maps to timeseries timestamp 
HOLIDAY_REGION = 'GB'   # Marks public holidays in the arima model

RESULTS_METRIC = 'mape' # smape, mape, rmse or mae

### Train Arima model up to a specific date 

The following query creates a model big query DATASET_NAME.MODEL_NAME
using data from DATASET_NAME.TABLE_NAME until TRAIN_UNTIL_DATETIME

In [ ]:
sql_forecast_build_model = """
CREATE OR REPLACE MODEL
  `{project}.{dataset}.{model}` OPTIONS(model_type='ARIMA',
    holiday_region='{holiday}',
    time_series_id_col='{ts_id_col}',
    time_series_data_col='{ts_data_col}',
    time_series_timestamp_col='{ts_timestamp_col}') AS
SELECT
  {ts_id_col},
  {ts_data_col},
  TIMESTAMP({ts_timestamp_col}) as {ts_timestamp_col}
FROM
  `{project}.{dataset}.{table}`
WHERE
  {ts_timestamp_col}<='{train_until}'
""".format(project=PROJECT_ID,
           dataset = DATASET_NAME, 
           model=MODEL_NAME, 
           table=TABLE_NAME, 
           holiday=HOLIDAY_REGION,
           ts_id_col=TIME_SERIES_ID_COLUMN, 
           ts_data_col=TIME_SERIES_DATA_COLUMN, 
           ts_timestamp_col=TIME_SERIES_TIMESTAMP_COLUMN,
           train_until=TRAIN_UNTIL_DATETIME)


query_job = client.query(sql_forecast_build_model)
query_job.result()

In [ ]:
print(sql_forecast_build_model)

### Forecast using the train model X datapoints in the future

We used the trained model DATASET_NAME.MODEL_NAME to forecast FORECAST_HORIZON (days) in the future
We then join the forcaset with the actual data DATASET_NAME.TABLE_NAME  in order to validate the predictions

In [ ]:
sql_forecast_horizon = """
SELECT 
   fc.* EXCEPT ({ts_id_col}),
   t.*
FROM
  ML.FORECAST(MODEL `{project}.{dataset}.{model}`,
  STRUCT({forecast_horizon} AS horizon)) AS fc
LEFT JOIN (
  SELECT
    *
  FROM
    `{project}.{dataset}.{table}`) AS t
ON
  CAST({ts_timestamp_col} AS TIMESTAMP) = fc.forecast_timestamp
  AND t.{ts_id_col}=fc.{ts_id_col}
""".format( project=PROJECT_ID,
            dataset = DATASET_NAME, 
           model=MODEL_NAME, 
           table=TABLE_NAME, 
           forecast_horizon=FORECAST_HORIZON, 
           ts_timestamp_col=TIME_SERIES_TIMESTAMP_COLUMN,
           ts_id_col=TIME_SERIES_ID_COLUMN,
          )

Pandas .read_gbq opperator is used to load the query results into a pandas dataframe

In [ ]:
print(sql_forecast_horizon)

In [ ]:
forecast = pd.read_gbq(sql_forecast_horizon, dialect='standard', use_bqstorage_api=True)
forecast.head()

In [ ]:
forecast = forecast.dropna()
forecast.index = forecast['forecast_timestamp']
id_list = forecast[TIME_SERIES_ID_COLUMN].unique().tolist()

In [ ]:
# Function to calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Function to calculate Symmetric MAPE - This solves issue with items that are sold 
# is small volumes skewing the results
def smape(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true[i]
        b = y_pred[i]
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    if out<0:
        return 100
    return out

# function to compute all evaluations metrics fiven actual and predictions
def compute_metrics(x):
    result = {
        'mae': mean_absolute_error(x[TIME_SERIES_DATA_COLUMN], x['forecast_value']), 
        'smape': smape(x[TIME_SERIES_DATA_COLUMN], x['forecast_value']), 
        'mape': mean_absolute_percentage_error(x[TIME_SERIES_DATA_COLUMN], x['forecast_value']), 
        'rmse': math.sqrt(mean_squared_error(x[TIME_SERIES_DATA_COLUMN], x['forecast_value']))}
    return pd.Series(result)


# Used to plot a list of items - actual and forecast
def plot_items(forecast, item_lst):
    row = 0
    col = 0
    columns_total = 3
    rows_total = math.ceil(len(item_lst)/columns_total)
    fig, ax =plt.subplots(rows_total,columns_total, figsize=(columns_total*8,rows_total*7))
    for i in item_lst:
        df = forecast[forecast[TIME_SERIES_ID_COLUMN] == i]
        sns.lineplot(x=TIME_SERIES_TIMESTAMP_COLUMN, y=TIME_SERIES_DATA_COLUMN,marker='o', color='deepskyblue', data=df,label="actual", linestyle="-", ax=ax[row,col%3]).set_title("Forecast for ts id: "+i)
        sns.lineplot(x=TIME_SERIES_TIMESTAMP_COLUMN, y="forecast_value",marker='o', color='mediumvioletred',ci=95, data=df,label="forecast", ax=ax[row,col%3])
        sns.lineplot(x=TIME_SERIES_TIMESTAMP_COLUMN, y="confidence_interval_upper_bound", color='pink', data=df, ax=ax[row,col%3])
        sns.lineplot(x=TIME_SERIES_TIMESTAMP_COLUMN, y="confidence_interval_lower_bound", color='pink', data=df, ax=ax[row,col%3])
        col+=1
        if(col%3==0):
            row+=1
            

### Calculate the error metrics Forecast vs Actual
We base our experiment on smape as it is the reasonable option from the available metrics. 
This is because different time series have different sale quantiti scale (tens and hundreds)

In [ ]:
result = forecast.groupby(TIME_SERIES_ID_COLUMN).apply(compute_metrics)
result= result.reset_index().sort_values(by=[RESULTS_METRIC])
result= result.replace([np.inf, -np.inf], np.nan).dropna()
print(' SMAPE | ID Count')
print('-------|-----------')
print('<10%   |  ',result[result[RESULTS_METRIC]<=10].count()[RESULTS_METRIC])
print('<20%   |  ',result[result[RESULTS_METRIC]<=20].count()[RESULTS_METRIC])
print('<30%   |  ',result[result[RESULTS_METRIC]<=30].count()[RESULTS_METRIC])
print('<40%   |  ',result[result[RESULTS_METRIC]<=40].count()[RESULTS_METRIC])
print('<50%   |  ',result[result[RESULTS_METRIC]<=50].count()[RESULTS_METRIC])
print('>50%   |  ',result[result[RESULTS_METRIC]>50].count()[RESULTS_METRIC])


print("\n\n All metrics")
result.mean()[1:]


This is how each threashold of MAPE can be interpreted <
<img src="mape-meaning.png" width=400 style="float:left">

### Pick the best and worst 20 items to show

In [ ]:
result = forecast.groupby(TIME_SERIES_ID_COLUMN).apply(compute_metrics)
result= result.reset_index().sort_values(by=[RESULTS_METRIC])
worst = result[-20:]
id_list_worst = worst[TIME_SERIES_ID_COLUMN].unique().tolist()
best = result[:20]
id_list_best = best[TIME_SERIES_ID_COLUMN].unique().tolist()

## Errors and Forecasts of the Most Accurate Time Series
<hr />

In [ ]:
plt.figure(figsize=(25, 4))
sns.barplot(x=TIME_SERIES_ID_COLUMN,y=RESULTS_METRIC, data=best).set_title('Most Accurate Predictions')

In [ ]:
plot_items(forecast, id_list_best)

## Errors and Forecasts of inacurate Time Series
<hr />

In [ ]:
plt.figure(figsize=(25, 4))
sns.barplot(x=TIME_SERIES_ID_COLUMN,y='smape', data=worst).set_title('Worst Predictions')

In [ ]:
plot_items(forecast, id_list_worst)

## Forecasts for all the Time Series
<hr />

In [ ]:
plot_items(forecast, id_list)